<a href="https://colab.research.google.com/github/meloly4/Assignment-2/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/meloly4/Assignment-2.git

Cloning into 'Assignment-2'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 17 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), done.


In [ ]:
!git clone https://github.com/huggingface/neuralcoref.git
!pip install -U spacy
!python -m spacy download en_core_web_lg

import spacy

%cd neuralcoref

!pip install -r requirements.txt
!pip install -e .

import neuralcoref

Cloning into 'neuralcoref'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 753 (delta 1), reused 2 (delta 0), pack-reused 748
Receiving objects: 100% (753/753), 67.82 MiB | 26.84 MiB/s, done.
Resolving deltas: 100% (398/398), done.
     |████████████████████████████████| 10.0MB 215kB/s 
     |████████████████████████████████| 2.1MB 31.7MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 782.7MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.3.1-cp36-none-any.whl size=782936125 sha256=c7b804b6978595e4631fb5acf16cd8ded68f8b3a921493e7511b50121c6db518
  Stored in directory: /tmp/pip-ephem-wheel-cache-3_4x99_0/wheels/ce/4d/1b/bc6cabb6df139c5f0318927

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 112 from PyObject
  return f(*args, **kwds)
100%|██████████| 40155833/40155833 [00:00<00:00, 53680562.51B/s]


In [ ]:
%cd /content/Assignment-2

/content/Assignment-2


In [ ]:
!python3 chains.py

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 112 from PyObject
  return f(*args, **kwds)


In [ ]:
!python3 example2.py

In [ ]:
#!/usr/bin/env python3

import pandas as pd
import neuralcoref
import spacy, json
import random, math
from collections import namedtuple, Counter, defaultdict as ddict
import tqdm
from pprint import pprint

# Choose your model here
# Recall that "en_core_web_sm" performed significantly worse with regards to coreference resolution
nlp = spacy.load("en_core_web_lg")
neuralcoref.add_to_pipe(nlp)


# This is a magic number
# When we are computing probabilitieis, we don't want anything to have occurred 0 times because some of our functions are undefined around 9
# To avoid 0's, we can use something called Plus One Smoothing, which simply assumes that there is at least a minimum amount of everything
# Obviously, this is not the most accurate thing, since there are almost certainly sentences that never have or will be said
# But it is very useful for estimating probabilities
PLUS_ONE_SMOOTHING = 0.01

ParsedStory = namedtuple("ParsedStory", "id title story one two three four five".split())

def load_data(rocstories):
    """Loads in rocstories csv file and iterates over the stories
    - Returns a generator of stories
    - a generator is like a collection/list that isn't populated until a item is requested
    - calling a ~next~ function on the generator gives an item if there is one left
    - if there are no items left, returns that it is empty
    """
    data = pd.read_csv(rocstories)
    return data.itertuples()

def parse_story(story):
    """Take a single story and run it through spacy"""
    name = story.storytitle
    # this is very compressed
    sentences = [nlp(sentence) for sentence in story[3:]]
    full_story = nlp(" ".join(story[3:]))
    return ParsedStory(story.storyid, name, full_story, *sentences)

# I've commented out this function to avoid confusion
# This was just a demonstration
#def parse_story_verbose(story):
#    """Same as parse_story but this is longer"""
#    name = story.storytitle
#    # sentences = [nlp(sentence) for sentence in story[3:]]
#    just_story = story[3:]
#    sentences = []
#    for sentence in just_story:
#        sentences.append(nlp(sentence))
#    # sentences is a list
#    full_story = nlp(" ".join(story[3:]))
#    return ParsedStory(name, full_story, sentences[0], sentences[1], sentences[2], sentences[3], sentences[4])


def take_sample(gen, sample=None, replacement=0.2):
    if not sample:
        """If we don't have a sample specified, return the entire generator"""
        return tqdm.tqdm(gen)
    # take the first ~sample~ number of stories
    # randomly replace stories as we iterate
    sentences = []

    for x in gen:
        sentences.append(x)
        if len(sentences) == sample:
            break
    # sentences has ~sample~ sentences in it
    for x in gen:
        if random.random() <= replacement:
            index = random.randint(1, sample) - 1
            sentences[index] = x
    return tqdm.tqdm(sentences)

def process_corpus(rocstories, sample=None, replacement=0.2):
    """rocstories is a string with the path to the rocstories.csv file
    sample: load a random sample of ~sample~ sentences
    """
    # 1 load the data
    # 2 iterate over everything
    #   3: parse everything
    data = load_data(rocstories)
    dataset = [parse_story(story) for story in take_sample(data, sample, replacement)]
    story_counter = ddict(list)

    for story in dataset:
        process_story(story, story_counter=story_counter)

    return dataset, ProbabilityTable(story_counter)

def process_story(parsedstory, heuristic=2, verbose=False, story_counter=ddict(list)):
    prot = protagonist(parsedstory, heuristic=2)
    parse_id, dep_pairs = extract_dependency_pairs(parsedstory)

    story_counter[parse_id] = dep_pairs

    if verbose:
        print("------------")
        print("Protagonist is: ", prot)
        print("story: ", parsedstory.title)
        for sentence in parsedstory[-5:]:
            print(sentence)
        for entity in dep_pairs:
            for d in dep_pairs[entity]:
                print("\t", d)
        print("------------")

# Return a per-story entity-id for a token
def dereference_pair(token, story):
    """returns a set id for an entity per story"""
    if token.text.lower() in ["i", "me"]:
        return -1
    pool = story._.coref_clusters
    for ref in pool:
        for mention in ref.mentions:
            if token == mention.root:
                return ref.i
    return None

#Return a list of dependency pairs
def extract_dependency_pairs(parse):
    """Get a story, return a list of dependency pairs"""
    verbs = [verb for verb in parse.story if verb.pos_ == "VERB"]
    deps = ddict(list)
    for verb in verbs:
        for child in verb.children:
            entity_index = dereference_pair(child, parse.story)
            # Add the word/dependency pair to the identified entity
            tup = (verb.lemma_, child.dep_)
            if entity_index != None:
                deps[entity_index].append(tup)
    return parse.id, deps

def coreferring_pairs(parse, token):
    """Because I'm nice, here's a function that gets all the dependency pairs that corefer to the given token
    This is inefficiently based on extract_dependency_pairs and dereference_pair for a reason.
    """
    extracted = extract_dependency_pairs(parse)
    res = dereference_pair(token, parse.story)
    if res is None:
        return []
    return extracted[res]

# Protagonist detection
def protagonist(story, heuristic=2):
    story = story.story
    if heuristic == 1:
        return protagonist_heuristic_one(story)
    elif heuristic == 2:
        return protagonist_heuristic_two(story)
    elif heuristic == 3:
        raise NotImplementedError

# Heuristic 1: first entity
def protagonist_heuristic_one(story):
    """Story is parsed by spacy"""
    return [(story.ents[0].text, 1)]

# Heuristic 2: most frequently mentioned entity
def protagonist_heuristic_two(story):
    #1 get entities
    if not story._.coref_clusters:
        return None
    return max(story._.coref_clusters, key=lambda cluster: len(cluster.mentions))

class ProbabilityTable:
    def __init__(self, counter):
        self.counter = counter
        self.cache = {}

    def write(self, fname="table.json"):
        with open(fname, 'w') as f:
            json.dump(self.counter, f, indent=2)

    def bigram(self, verb, dependency, verb2, dependency2):
        """Find all the stories where story contains verb,dependency and verb2,dependency2
        AND they refer to the same entity
        """
        query = (verb, dependency, verb2, dependency2)
        if query not in self.cache:
            ctr = 0
            for story in self.counter:
                for entity in self.counter[story]:
                    v = self.counter[story][entity]
                    if [verb, dependency] in v and [verb2, dependency2] in v:
                        ctr +=1
            self.cache[query] = ctr
        return self.cache[query]

    def unigram(self, verb, dependency):
        """Number of stories containing verb/dependency"""
        query = (verb, dependency)
        if query not in self.cache:
            ctr = 0
            for story in self.counter:
                for entity in self.counter[story]:
                    if [verb, dependency] in self.counter[story][entity]:
                        ctr += 1
            self.cache[query] = ctr
        return self.cache[query]

    def pmi(self, verb, dependency, verb2, dependency2):
        n = len(self.counter) + PLUS_ONE_SMOOTHING
        prob_a_and_b = (self.bigram(verb, dependency, verb2, dependency2)+PLUS_ONE_SMOOTHING)/n
        prob_a = (self.unigram(verb, dependency)+PLUS_ONE_SMOOTHING)/n
        prob_b = (self.unigram(verb2, dependency2)+PLUS_ONE_SMOOTHING)/n
        return math.log(prob_a_and_b/(prob_a*prob_b))
        #math.log(prob_a_and_b) - (math.log(prob_a) + math.log(prob_b))

    def histo(self, verb, dependency):
        """Return cooccurrence counts for all verb/dependency pairs for a given verb/dependency"""
        ctr = Counter()
        for story in self.counter:
            for entity in self.counter[story]:
                if (verb, dependency) in self.counter[story][entity]:
                    for c in self.counter[story][entity]:
                        if c != (verb, dependency):
                            ctr[c] += 1
        return ctr


    def histo_pmi(self, verb, dependency):
        return sorted([(v, d, self.pmi(verb, dependency, v, d)) for v, d in self.histo(verb, dependency)], key=lambda x: x[-1])

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 112 from PyObject
  return f(*args, **kwds)


In [ ]:
import chains
import json
from pprint import pprint


def parse_test_instance(story):
    """Returns TWO ParsedStory instances representing option 1 and 2"""
    # this is very compressed
    id = story.InputStoryid
    story = list(story)
    sentences = [chains.nlp(sentence) for sentence in story[2:6]]
    alternatives = [story[6], story[7]]
    return [chains.ParsedStory(id, id, chains.nlp(" ".join(story[2:6]+[a])), *(sentences+[chains.nlp(a)])) for a in alternatives]

def story_answer(story):
    """Tells you the correct answer. Return (storyid, index). 1 for the first ending, 2 for the second ending"""
    #obviously you can't use this information until you've chosen your answer!
    return story.InputStoryid

# Load training data and build the model
# data, table = chains.process_corpus("train.csv", 100)
# print(table.pmi("move", "nsubj", "move", "nsubj"))

# load the pre-built model
with open("all.json") as fp:
    table = chains.ProbabilityTable(json.load(fp))

# load testing data
test = chains.load_data("tinytest.csv")
for t in test:
    one, two = parse_test_instance(t)
    one_deps = chains.extract_dependency_pairs(one)
    pprint(one[2:])
    pprint(two[2:])
    # logic to choose between one and two
    pprint("answer:"+ str(story_answer(t)))
    # pprint(t[6])
    # for key,value in one_deps[1].items():
    #   for v in value:
        # pprint(table.pmi(v[0],v[1],v[0],v[1]))
        # pprint(table.bigram(v[0],v[1],v[0],v[1]))
    # pprint(t[7])
    # for key,value in two_deps[1].items():
    #   for v in value:
    #     pprint(v[0])
    #     pprint(table.pmi(v[0],v[1],v[0],v[1]))
    #     pprint(table.bigram(v[0],v[1],v[0],v[1]))

In [ ]:
import chains
import json
from pprint import pprint


def parse_test_instance(story):
    """Returns TWO ParsedStory instances representing option 1 and 2"""
    # this is very compressed
    id = story.InputStoryid
    story = list(story)
    sentences = [chains.nlp(sentence) for sentence in story[2:6]]
    alternatives = [story[6], story[7]]
    return [chains.ParsedStory(id, id, chains.nlp(" ".join(story[2:6]+[a])), *(sentences+[chains.nlp(a)])) for a in alternatives]

# def story_answer(story):
#     """Tells you the correct answer. Return (storyid, index). 1 for the first ending, 2 for the second ending"""
#     #obviously you can't use this information until you've chosen your answer!
#     return story.InputStoryid

# Load training data and build the model
# data, table = chains.process_corpus("train.csv", 100)
# print(table.pmi("move", "nsubj", "move", "nsubj"))

# load the pre-built model
with open("all.json") as fp:
    table = chains.ProbabilityTable(json.load(fp))

# load testing data
test = chains.load_data("tinytest.csv")
for t in test:
    one, two = parse_test_instance(t)
    one_deps = chains.extract_dependency_pairs(one)
    two_deps = chains.extract_dependency_pairs(two)
    pprint(one_deps)
    # pprint(chains.coreferring_pairs(one,protagonist(one).main.root))



('138d5bfb-05cc-41e3-bf2c-fa85ebad14e2',
 defaultdict(<class 'list'>,
             {0: [('grow', 'nsubj'),
                  ('find', 'nsubj'),
                  ('shoot', 'nsubjpass'),
                  ('turn', 'nsubj')]}))
('bff9f820-9605-4875-b9af-fe6f14d04256',
 defaultdict(<class 'list'>,
             {0: [('need', 'nsubj'),
                  ('decide', 'nsubj'),
                  ('choose', 'nsubj'),
                  ('test', 'nsubj'),
                  ('eat', 'nsubj')],
              1: [('choose', 'dobj'), ('follow', 'dobj')]}))
('e8f628d5-9f97-40ed-8611-fc0e774673c4',
 defaultdict(<class 'list'>,
             {0: [('dream', 'nsubj'),
                  ('save', 'nsubj'),
                  ('land', 'nsubj'),
                  ('like', 'nsubj'),
                  ('decide', 'nsubj')],
              1: [('visit', 'dobj')]}))
('f5226bfe-9f26-4377-b05f-3d9568dbdec1',
 defaultdict(<class 'list'>,
             {0: [('intend', 'nsubj'), ('like', 'nsubj'), ('eat', 'nsubj')],
        